In [1]:
from lightfm_recomender import LightfmRecomender
import os
%load_ext autoreload
%autoreload 2
new_directory = '/Users/vasevooo/projects/GameBuddy'
os.chdir(new_directory)
import pandas as pd 
import requests
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from lightfm import LightFM
from scipy.sparse import csr_matrix, save_npz, load_npz

/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### обновление предобработанных данных для модели 

In [93]:
## Загрузить файл на свой локальный диск, если еще нет

file_url = 'https://drive.google.com/file/d/1tt1DxIxKtJueQd81Wa6XibM1vDuuje9o/view?usp=sharing' ## Ссылка на гугл драйв от Вани

In [10]:
ru_ids = pd.read_csv('data/final_gametime_df.csv')

In [11]:
transformed_df = pd.melt(ru_ids, id_vars='steamid', var_name='game_id', value_name='ranking')
transformed_df['game_id'] = transformed_df['game_id'].astype(int)

In [12]:
scaler = MinMaxScaler(feature_range=(0, 100))

transformed_df['s_ranking'] = transformed_df.groupby('game_id')['ranking'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())
transformed_df['s_ranking'] = pd.to_numeric(transformed_df['s_ranking'])

In [94]:
## сохранение df по юзерам для дальнейшего использования

transformed_df.to_csv('data/for_models/users_games_df.csv', index=False)

In [16]:
games = pd.read_csv('data/game_info_with_scores.csv')
item_dict ={}
games = games[['appid', 'Name']].sort_values('appid').reset_index()
games['appid'] = games['appid'].astype(int)

for i in range(games.shape[0]):
    item_dict[int(games.loc[i,'appid'])] = games.loc[i,'Name']

In [96]:
## сохранение словаря по играм для дальнейшего использования

with open("data/for_models/item_dict.json", "w") as outfile:
    json.dump(item_dict, outfile, indent=4)

### Тест класса LigthFM Recomender

In [101]:
item_dict_path = 'data/for_models/item_dict.json'
users_games_df_path = 'data/for_models/users_games_df.csv'

lfr = LightfmRecomender(users_games_df_path, item_dict_path)

In [4]:
steamid = 76561198156258305

In [29]:
new_user = lfr.get_user_games(steamid)

In [30]:
matrix, df = lfr.get_csr_matrix_for_all_users(new_user)

In [31]:
model = lfr.fit_model(matrix, epochs=1)

In [32]:
lfr.sample_recommendation_user(model, df, steamid, threshold = 0, nrec_items = 10)

User: 76561198156258305
Recommended Items:
1- Terraria
2- Portal 2
3- Don't Starve Together
4- Garry's Mod
5- Dying Light
6- Tomb Raider
7- The Witcher 2: Assassins of Kings Enhanced Edition
8- The Elder Scrolls V: Skyrim
9- Half-Life 2
10- Grand Theft Auto V

 Known Likes:
1- Torchlight: Infinite
2- EA SPORTS™ FIFA 23
3- Vampire Survivors
4- V Rising
5- God of War
6- The Callisto Protocol™
7- Forza Horizon 4
8- ELDEN RING
9- Gunfire Reborn
10- Potion Craft: Alchemist Simulator
11- Gears Tactics
12- Sea of Thieves 2023 Edition
13- Apex Legends™
14- Total War: WARHAMMER III
15- Curse of the Dead Gods
16- Cyberpunk 2077
17- Destiny 2
18- Ori and the Will of the Wisps
19- Dota Underlords
20- Mortal Kombat 11
21- Among Us
22- Back 4 Blood
23- Age of Empires II: Definitive Edition
24- DOOM Eternal
25- Gloomhaven
26- Ring of Elysium
27- UnderMine
28- Slay the Spire
29- Disco Elysium - The Final Cut
30- Quake Champions
31- Realm Grinder
32- Graveyard Keeper
33- Hunt: Showdown
34- Dead Cells
3

In [102]:
df_cold_start = pd.DataFrame({'game_names': ['Counter-Strike', 'Team Fortress Classic', 'Day of Defeat'],
                   'rankings': [5, 5, 8]})

In [ ]:
item_dict

In [103]:
cold_start_user = lfr.get_user_games_cold_start(df_cold_start)

In [104]:
cold_start_user.sort_values(by='ranking', ascending=False).head(10)

,game_id,game_name,ranking,s_ranking,steam_id
2,30,Day of Defeat,8.0,1.414214,1234567891010102930
0,10,Counter-Strike,5.0,-0.707107,1234567891010102930
1,20,Team Fortress Classic,5.0,-0.707107,1234567891010102930
1783,2221490,0,0.0,0.000000,1234567891010102930
1186,550650,0,0.0,0.000000,1234567891010102930
1197,557400,0,0.0,0.000000,1234567891010102930
1196,557340,0,0.0,0.000000,1234567891010102930
1195,555570,0,0.0,0.000000,1234567891010102930
1194,555160,0,0.0,0.000000,1234567891010102930
1193,554620,0,0.0,0.000000,1234567891010102930


In [90]:
cold_start_user[cold_start_user['game_id'] == 10]

,game_id,game_name,ranking,steam_id
0,10,Counter-Strike,0.0,1234567891010102930


In [68]:
matrix, df = lfr.get_csr_matrix_for_all_users(cold_start_user)

KeyboardInterrupt: 